## ResFPN Classifier Tutorial - COVIDx

by *Ming Ming Zhang*

In [1]:
import os, sys

# python files directory
PY_DIR = #'directory/to/python/files'
sys.path.append(PY_DIR)

import covidx, resnet_fpn

### Preprocess Data

In [2]:
dataset_dir = #'directory/to/covidx'

covidx.move_imgs(dataset_dir, subset='train')
covidx.move_imgs(dataset_dir, subset='test')

Processed train 0.7 seconds
Processed test 0.1 seconds


In [3]:
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

# for the first time use, set convert_rgb to True
convert_rgb = False
if convert_rgb:
    covidx.covert_rgb_dir(train_dir)
    covidx.covert_rgb_dir(test_dir)

In [4]:
batch_size = 32
image_size = (256,256)
train_ds, val_ds, test_ds, classes = covidx.load_data(
    dataset_dir, batch_size, image_size)

Found 13917 files belonging to 3 classes.
Using 11134 files for training.
Found 13917 files belonging to 3 classes.
Using 2783 files for validation.
Found 1579 files belonging to 3 classes.


In [5]:
total = 13917
class_weight = []
for class_name in classes:
    imgs_list = os.listdir(os.path.join(train_dir, class_name))
    num_imgs = len(imgs_list)
    weight = (1/num_imgs)*(total/len(classes))
    class_weight.append(weight)
    print('%s: %d, ratio=%.4f' \
          % (
        class_name, 
        num_imgs, 
        num_imgs/total
    ))

COVID-19: 489, ratio=0.0351
normal: 7965, ratio=0.5723
pneumonia: 5463, ratio=0.3925


In [6]:
class_weight = [x/sum(class_weight) for x in class_weight]
for i in range(len(classes)):
    print('%s: weight=%f' % (classes[i], class_weight[i]))

COVID-19: weight=0.868882
normal: weight=0.053344
pneumonia: weight=0.077775


### ResFPN Classifiers

In [7]:
image_shape = (256,256,3)
filepath = #'path\to\pretrained_resnet_weights.h5'

#### Cross-entropy Loss

In [26]:
ResFPN_ce = resnet_fpn.ResFPN_Classifier(
    image_shape=image_shape, 
    num_classes=len(classes), 
    num_filters=256, 
    architecture='resnet50', 
    augmentation=False,
    checkpoint_path=None,
    resnet_weights_path=filepath)

ResFPN_ce.train(
    train_dataset=train_ds, 
    val_dataset=val_ds, 
    params={'lr':0.01, 'l2':0.1, 'epochs':5},
    loss_type='ce',
    save_weights=False)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
348/348 [==============================] - ETA: 0s - loss: 12.7869 - dense_resnet_loss: 0.3960 - dense_fpn_2_loss: 0.5644 - dense_fpn_3_loss: 0.5110 - dense_fpn_4_loss: 0.4451 - dense_fpn_5_loss: 0.4337 - dense_resnet_accuracy: 0.8585 - dense_fpn_2_accuracy: 0.8234 - dense_fpn_3_accuracy: 0.8360 - dense_fpn_4_accuracy: 0.8517 - dense_fpn_5_accuracy: 0.8502WARNING:te

In [27]:
top_idxes, ensemble_acc = ResFPN_ce.select_top(val_ds, top=3)

87/87 [==============================] - 10s 113ms/step - loss: 11.5226 - dense_resnet_loss: 0.2082 - dense_fpn_2_loss: 0.2364 - dense_fpn_3_loss: 0.2273 - dense_fpn_4_loss: 0.2093 - dense_fpn_5_loss: 0.2047 - dense_resnet_accuracy: 0.9263 - dense_fpn_2_accuracy: 0.9224 - dense_fpn_3_accuracy: 0.9249 - dense_fpn_4_accuracy: 0.9281 - dense_fpn_5_accuracy: 0.9263

Top classifiers: ['res_fpn_5', 'resnet', 'res_fpn_4']
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @

In [28]:
ensemble_class_ids, metrics, f1_score = ResFPN_ce.predict(
    test_ds, classes, display_metrics=True, top_idxes=top_idxes)


Test accuracy: 0.92

COVID-19:
  precision 0.65, recall 0.24, F1-score 0.35
normal:
  precision 0.93, recall 0.92, F1-score 0.92
pneumonia:
  precision 0.88, recall 0.91, F1-score 0.89

Average:
  precision 0.82, recall 0.69, F1-score 0.72


In [34]:
import numpy as np
f1s_ce = np.array([0.70,0.72,0.73,0.72,0.75])
print('ce: mean %.2f, std %.2f' %(np.mean(f1s_ce), np.std(f1s_ce)))

ce: mean 0.72, std 0.02


#### Focal Loss

In [8]:
alphas=[class_weight]
gammas=[0,1,2,5]

best_f1, best_alpha, best_gamma = covidx.tune_focal(
    train_ds, val_ds, classes, alphas, gammas, image_shape, filepath)

alpha [0.8688815552887377, 0.05334376403467579, 0.07777468067658662]


gamma: 0
Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.exper


Validation accuracy: 0.03556034482758621

Test accuracy: 0.39

COVID-19:
  precision 0.00, recall 0.00, F1-score 0.00
normal:
  precision 0.00, recall 0.00, F1-score 0.00
pneumonia:
  precision 0.39, recall 1.00, F1-score 0.56

Average:
  precision 0.13, recall 0.33, F1-score 0.19


gamma: 1
Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
348/348 [==============================] - ETA: 0s - loss: 10.6450 - dense

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
348/348 [==============================] - ETA: 0s - loss: 10.6192 - dense_resnet_loss: 0.0230 - dense_fpn_2_loss: 0.0511 - dense_fpn_3_loss: 0.0453 - dense_fpn_4_loss: 0.0359 - dense_fpn_5_loss: 0.0272 - dense_resnet_accuracy: 0.7765 - dense_fpn_2_accuracy: 0.7461 - dense_fpn_3_accuracy: 0.7465 - dense_fpn_4_accuracy: 0.7601 - dense_fpn_5_accuracy: 0.7802WARNING:te

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
348/348 [==============================] - ETA: 0s - loss: 10.5463 - dense_resnet_loss: 0.0098 - dense_fpn_2_loss: 0.0355 - dense_fpn_3_loss: 0.0330 - dense_fpn_4_loss: 0.0194 - dense_fpn_5_loss: 0.0120 - dense_resnet_accuracy: 0.7000 - dense_fpn_2_accuracy: 0.7008 - dense_fpn_3_accuracy: 0.7050 - dense_fpn_4_accuracy: 0.7288 - dense_fpn_5_accuracy: 0.7191WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000001A1123C5168> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments'

In [29]:
ResFPN_focal = resnet_fpn.ResFPN_Classifier(
    image_shape=image_shape, 
    num_classes=len(classes), 
    num_filters=256, 
    architecture='resnet50', 
    augmentation=False,
    checkpoint_path=None,
    resnet_weights_path=filepath)

ResFPN_focal.train(
    train_dataset=train_ds, 
    val_dataset=val_ds, 
    params={'lr':0.01, 'l2':0.1, 'epochs':5, 
            'alpha':class_weight, 'gamma':2.0},
            #'alpha':best_alpha, 'gamma':best_gamma},
    loss_type='focal',
    save_weights=False)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
348/348 [==============================] - ETA: 0s - loss: 10.6246 - dense_resnet_loss: 0.0218 - dense_fpn_2_loss: 0.0564 - dense_fpn_3_loss: 0.0467 - dense_fpn_4_loss: 0.0353 - dense_fpn_5_loss: 0.0278 - dense_resnet_accuracy: 0.7773 - dense_fpn_2_accuracy: 0.7076 - dense_fpn_3_accuracy: 0.7237 - dense_fpn_4_accuracy: 0.7441 - dense_fpn_5_accuracy: 0.7592WARNING:te

In [30]:
top_idxes, ensemble_acc = ResFPN_focal.select_top(val_ds, top=3)

87/87 [==============================] - 10s 113ms/step - loss: 10.5131 - dense_resnet_loss: 0.0148 - dense_fpn_2_loss: 0.0149 - dense_fpn_3_loss: 0.0152 - dense_fpn_4_loss: 0.0158 - dense_fpn_5_loss: 0.0157 - dense_resnet_accuracy: 0.9202 - dense_fpn_2_accuracy: 0.9249 - dense_fpn_3_accuracy: 0.9256 - dense_fpn_4_accuracy: 0.9253 - dense_fpn_5_accuracy: 0.9181

Top classifiers: ['resnet', 'res_fpn_2', 'res_fpn_3']
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @

In [31]:
ensemble_class_ids, metrics, f1_score = ResFPN_focal.predict(
    test_ds, classes, display_metrics=True, top_idxes=top_idxes)


Test accuracy: 0.92

COVID-19:
  precision 0.82, recall 0.39, F1-score 0.50
normal:
  precision 0.94, recall 0.93, F1-score 0.93
pneumonia:
  precision 0.89, recall 0.91, F1-score 0.90

Average:
  precision 0.88, recall 0.74, F1-score 0.78


In [36]:
f1s_focal = np.array([0.77,0.78,0.77,0.76,0.78])
print('focal: mean %.2f, std %.2f' %(np.mean(f1s_focal), np.std(f1s_focal)))

focal: mean 0.77, std 0.01
